In [ ]:
!pip install tensorflow-gpu==2.0.0-beta1

In [ ]:
# import
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, UpSampling2D, Input, Conv2DTranspose
from tensorflow.keras.models import Model
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
import seaborn as sns
import pandas as pd
import numpy as np
import os
%matplotlib inline
print(tf.__version__)
print(tf.keras.__version__)

In [ ]:
os.listdir('../input/ml2019spring-hw7/images')

In [ ]:
# load Data
file_PATH = '../input/ml2019spring-hw7/images/images'

def load_data(file_PATH):
    file_list = []
    file_list = os.listdir(file_PATH)
    data = []
    for file in file_list:
        tmp = Image.open(os.path.join(file_PATH, file))
        data.append(np.array(tmp))
    data = np.array(data)
    data_shape = data.shape
    print(data.shape)
    return data, data_shape


data, data_shape = load_data(file_PATH)
nor_data = data / 255
data_shape[1:]


In [ ]:
# build model
input = Input(data_shape[1:])

# encoding
x = Conv2D(32, (3, 3), padding='same', activation='relu')(input)  # 32,32,32
x = MaxPooling2D(2, 2, padding='same')(x)  # 16,16,32
x = Conv2D(8, (3, 3), padding='same', activation='relu')(x)  # 16,16,8
x = MaxPooling2D(2, 2, padding='same')(x)  # 8,8,8

encoded = Conv2D(1, (3, 3), padding='same', activation='relu')(x)  # 8,8,8
# MaxPooling2D(2, 2, padding='same')(x)

# decodeing
x = Conv2DTranspose(1, (3, 3), padding='same', activation='relu')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2DTranspose(8, (3, 3), padding='same', activation='relu')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2DTranspose(32, (3, 3), padding='same', activation='relu')(x)
# x = UpSampling2D((2, 2))(x)
decoded = Conv2DTranspose(3, (3, 3), padding='same', activation='relu')(x)


In [ ]:
autoencoder = Model(input, decoded)
plot_model(autoencoder, show_shapes=True)

In [ ]:
# model compile


autoencoder.compile(optimizer='Adam', loss='mse')
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=5, verbose=1, mode='auto')
autoencoder_fit = autoencoder.fit(nor_data, nor_data, batch_size=100, epochs=200, callbacks=[callback])

In [ ]:
# saving model
autoencoder.save('autoencoder.h5')

In [ ]:
# building encoding model
encoder = Model(input, encoded)
code = encoder.predict(nor_data)
code = code.reshape((-1,64))


In [ ]:
code.shape

In [ ]:
# dimention reduction
tsne = TSNE(n_components=2,init='pca',n_iter=3000,perplexity=20)
tsne_fit = tsne.fit_transform(code)

In [ ]:
pca = PCA(n_components=2, copy=False, whiten=True, svd_solver='full')
pca_fit = pca.fit_transform(code)

In [ ]:
tsne_df = pd.DataFrame(tsne_fit,columns=['x','y'])
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="x", y="y",

    data=tsne_df,
    legend="full"
)

In [ ]:
pca_df = pd.DataFrame(pca_fit,columns=['x','y'])
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="x", y="y",

    data=pca_df ,
    legend="full"
)

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=501).fit(tsne_fit)

In [ ]:
pred =kmeans.labels_
tsne_df['label']=pred

In [ ]:
tsne_df.head()

In [ ]:
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="x", y="y",hue='label',
    data=tsne_df,
    legend="full"
)

In [ ]:
test = np.genfromtxt('../input/ml2019spring-hw7/test_case.csv',delimiter=',',skip_header=1,dtype = int)

In [ ]:
result=[]

for row in range(len(test)):
    pred1, pred2 = pred[test[row][1]-1],pred[test[row][2]-1]
    if pred1 == pred2:
        result.append([row,1])
    else:
        result.append([row,0])
    

In [ ]:
result_df=pd.DataFrame(result,columns=['id','label'])
result_df.to_csv('submission.csv',index=False)